In [34]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.\
        builder.\
        appName("streamingExampleRead").\
        config("spark.mongodb.read.connection.uri", "mongodb://127.0.0.1/mini.test?readPreference=primaryPreferred").\
        config("spark.mongodb.write.connection.uri", "mongodb://127.0.0.1/mini.test").\
        config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:10.4.0').\
        getOrCreate()

In [5]:
dataFrame = spark.read\
                 .format("mongodb")\
                 .option("database", "mini")\
                 .option("collection", "test")\
                 .load()

In [8]:
dataFrame.printSchema()

root
 |-- _id: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



In [6]:
dataFrame.show()

+--------------------+---+------+
|                 _id|age|  name|
+--------------------+---+------+
|66e9cc1a44143d0c7...| 23|BooVer|
+--------------------+---+------+



In [49]:
# define the schema of the source collection
readSchema = (StructType()
  .add('_id', StringType())
  .add('title', StringType())
  #.add('age', IntegerType())
)

# define a streaming query
streamingDataFrame = (spark.readStream
  .format("mongodb")
  .option("spark.mongodb.connection.uri", "mongodb://localhost:27017")
  .option('spark.mongodb.database', "mini")
  .option('spark.mongodb.collection', "yt")
  .schema(readSchema)
  .load()
)

  # manipulate your streaming data
dataStreamWriter = (streamingDataFrame  
  .writeStream
  .trigger(continuous="1 second")
  .format("memory")
  .queryName("myMemoryTable")
  .outputMode("append")
)
# run the query
query = dataStreamWriter.start()

query.awaitTermination()

StreamingQueryException: [STREAM_FAILED] Query [id = 020b5e6a-4c16-45a2-a813-8c63b13100ce, runId = b655befc-e1ca-454b-866f-140361edceb9] terminated with exception: Job aborted due to stage failure: Task 0 in stage 20.0 failed 1 times, most recent failure: Lost task 0.0 in stage 20.0 (TID 20) (DESKTOP-57VRIHT executor driver): org.apache.spark.SparkException: Data read failed.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedToReadDataError(QueryExecutionErrors.scala:2243)
	at org.apache.spark.sql.execution.streaming.continuous.ContinuousQueuedDataReader.next(ContinuousQueuedDataReader.scala:106)
	at org.apache.spark.sql.execution.streaming.continuous.ContinuousDataSourceRDD$$anon$1.getNext(ContinuousDataSourceRDD.scala:102)
	at org.apache.spark.sql.execution.streaming.continuous.ContinuousDataSourceRDD$$anon$1.getNext(ContinuousDataSourceRDD.scala:94)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at scala.collection.Iterator$$anon$10.hasNext(It